In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_mldata as ldata
from sklearn.cross_validation import train_test_split
from sklearn import datasets
from scipy.linalg import solve
from scipy.sparse import linalg, eye
from scipy import stats
from scipy.spatial import distance
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import random
import math

/Users/melkigabriel/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
movie = pd.read_csv('/Users/melkigabriel/Downloads/ml-100k/u5.base',
                    sep='\t',
                    names =['user_id','item_id','rating','timestamp'])
movie.head()

ratings= pd.read_csv('/Users/melkigabriel/Downloads/ml-1m/ratings.dat',
                    sep='::',
                    names =['user_id','item_id','rating','timestamp'])

movie = movie
t = pd.crosstab(movie['user_id'],  movie['item_id']) #Matrice de filtre, =1 s'il y a un rating, zero sinon

for i in range(len(movie)):
    t[int(movie[i:(i+1)]['item_id'])][int(movie[i:(i+1)]['user_id'])]=movie[i:(i+1)]['rating']


/Users/melkigabriel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [133]:
class CollaborativeFiltering(object):
    
    def __init__(self, dimFactorisation = 5, reg = 0.001, bias = False):
        
        self.dimFactorisation = dimFactorisation
        self.loss_histo = []
        self.reg = reg
        self.bias = bias
        self.loss_histo = []
        
    def fit(self, users, items, ratings, nb_epochs = 100, learning_rate=0.01, verbose=True):
        #Initialisation des deux matrices U et V  (x = u*v)
        self.users = users
        self.items = items
        nbUsers = max(users)
        nbItems = max(items)
        
        
        self.u = np.random.rand(self.dimFactorisation, nbUsers)
        self.v = np.random.rand(self.dimFactorisation,nbItems)
        
        #Initialisation eventuelle des biais
        if self.bias :
            self.usersBias = np.random.rand(nbUsers)
            self.itemsBias = np.random.rand(nbItems)
            self.globalBias = np.random.rand(1)[0]
        
        #Nombre de notes 
        nbRatings = len(ratings)
        
        for ep in range(nb_epochs):
            loss = 0

            #Random index
            self.ratingIndex = np.random.randint(nbRatings) #Rating tire au hasard

            #Utilisateur/item  séléctionnés
            self.user_id = int(users[self.ratingIndex]) - 1
            self.item_id = int(items[self.ratingIndex]) - 1
            self.rating_id = ratings[self.ratingIndex]
            
            #Estimation actuelle du rating
            prediction = self.u[:, self.user_id].dot(self.v[:,self.item_id])
            
            #Ajout eventuel des biais
            if self.bias : 
                prediction += self.globalBias + self.usersBias[self.user_id] + self.itemsBias[self.item_id]

            diff = (self.rating_id - prediction)

            loss +=diff**2

            factor = 1-self.reg*learning_rate
            self.u[:,self.user_id] = factor * self.u[:,self.user_id] + learning_rate * self.v[:,self.item_id] * diff
            self.v[:,self.item_id] = factor * self.v[:,self.item_id] + learning_rate * self.u[:,self.user_id] * diff

            if self.bias:
                self.usersBias[self.user_id] = factor * self.usersBias[self.user_id] + learning_rate * diff
                self.itemsBias[self.item_id] = factor * self.itemsBias[self.item_id] + learning_rate * diff
                self.globalBias = factor * self.globalBias + learning_rate * diff

            self.loss_histo.append(loss)
            if verbose :
                if(ep%int(nb_epochs/10)==0):
                    accuracy_train = np.array([round(self.u[:,(users[i]-1)].dot(self.v[:,(items[i]-1)])) == ratings[i] 
                                   for i in range(nbRatings)])
                    print("Accuracy : %d : %f"%(ep, np.mean(accuracy_train)))

        print("Apprentissage du modele fini, accuracy final en train : %f"%(np.mean(accuracy_train)))
        
        
   # round_vecto = np.vectorize(round)
    
    def predict(self, userIds, itemIds):
        preds =[]
        for i in range(len(userIds)):   
            if itemIds[i] not in self.items :
                if userIds[i] not in self.users:
                    preds.append(np.random.randint(1,5))
                else : 
                    preds.append(round((np.random.rand(1,self.dimFactorisation).dot(self.v[:,itemIds[i]-1]))[0]))
           
            elif userIds[i] not in self.users:
                preds.append(round(np.random.rand(1,self.dimFactorisation).dot(self.u[:,userIds[i]-1])[0]))

            else :
                preds.append(round((self.u[:,(userIds[i]-1)].dot(self.v[:,(itemIds[i]-1)]))))
        
        return np.array(preds)
        
    def score(self, userIds, itemIds, ratings):
        preds = self.predict(userIds, itemIds)
        return np.mean(preds.reshape(ratings.shape) == ratings)

In [134]:
user = np.array(movie['user_id']) 
item = np.array(movie['item_id']) 
ratings = np.array(movie['rating'])
timestamps = np.array(movie['timestamp'])


user_train, user_test, item_train, item_test, ratings_train, ratings_test = train_test_split(
            user, item, ratings, train_size=0.8)


cf = CollaborativeFiltering(bias=False)
cf.fit(user_train, item_train, ratings_train, nb_epochs=50000)

print("L'accuracy en test est de  "+
      str(1*(cf.predict(user_test,item_test)==ratings_test).mean()))


cfBias = CollaborativeFiltering(bias=True)
cfBias.fit(user_train, item_train, ratings_train, nb_epochs=50000)

print("L'accuracy en test est de  "+
      str(1*(cfBias.predict(user_test,item_test)==ratings_test).mean()))


Accuracy : 0 : 0.074688
Accuracy : 5000 : 0.113672
Accuracy : 10000 : 0.171875
Accuracy : 15000 : 0.229109
Accuracy : 20000 : 0.270078
Accuracy : 25000 : 0.298000
Accuracy : 30000 : 0.316391
Accuracy : 35000 : 0.328703
Accuracy : 40000 : 0.342469
Accuracy : 45000 : 0.350750
Apprentissage du modele fini, accuracy final en train : 0.350750
Accuracy : 0 : 0.072516
Accuracy : 5000 : 0.069875
Accuracy : 10000 : 0.064281
Accuracy : 15000 : 0.062344
Accuracy : 20000 : 0.061281
Accuracy : 25000 : 0.060812
Accuracy : 30000 : 0.061469
Accuracy : 35000 : 0.060078
Accuracy : 40000 : 0.059859
Accuracy : 45000 : 0.060234
Apprentissage du modele fini, accuracy final en train : 0.060234
L'accuracy en test est de  0.3461875
L'accuracy en test est de  0.05975


In [94]:
def loadLinks(path):
    d={}
    with open(path) as f:
        for l in f:
            w = l.split()
            d[w[0]]= w[1:]
    return(d)        

In [95]:
links = loadLinks('/Users/melkigabriel/Desktop/UPMC/FDMS-2/recodata/u.links')

In [96]:
m_links = np.zeros((944,944))
for i in links.keys():
    for k in links[str(i)]:
        m_links[int(i),int(k)]=1     

In [216]:
def g(x):
    return(4/(1+math.exp(-x)))

def g_prime(x):
    return(g(x)*(1-g(x)/4))

#g = np.vectorize(g)
#g_prime = np.vectorize(g_prime)

In [217]:
class SoRec(object):
    
    def __init__(self, dimFactorisation = 5, bias = False,
                    lambda_c=0.1, lambda_u=0.1, lambda_v=0.1, lambda_z=0.1,
                    nb_epochs = 1000, learning_rate=0.1):

        self.dimFactorisation = dimFactorisation
        self.loss_histo = []
        self.bias = bias
        self.loss_histo = []
        
        
    def fit(self, users, items, ratings, links_data):
        
        #Initialisation des deux matrices U et V  (x = u*v)
        self.users = users
        self.items = items
        
        nbUsers = max(users)
        nbItems = max(items)
        
        #a mettre dans les hyperparametres
        # U est de dimensions K x NbUsers
        self.u = 5*(np.random.rand(self.dimFactorisation, nbUsers)-0.5)
        
        # Z est de dimensions K x NbUsers
        self.z = 5*(np.random.rand(self.dimFactorisation, nbUsers)-0.5)
        
        #V est de dimensions NbItems x K
        self.v = 5*(np.random.rand(self.dimFactorisation, nbItems)-0.5)
        
        #Initialisation eventuelle des biais
        if self.bias :
            self.usersBias = np.random.rand(nbUsers)
            self.itemsBias = np.random.rand(nbItems)
            self.globalBias = np.random.rand(1)[0]
        
        #Nombre de notes 
        nbRatings = len(ratings)
        
        #SGD
        for ep in range(nb_epochs):
            loss = 0
            
            #Random index
            self.ratingIndex = np.random.randint(nbRatings) #Rating tire au hasard
            self.index_z = np.random.randint(nbUsers)

            #Utilisateur/item  séléctionnés
            self.user_id = int(users[self.ratingIndex]) - 1
            self.item_id = int(items[self.ratingIndex]) - 1
            self.rating_id = ratings[self.ratingIndex]
            self.link_id = links_data[self.user_id,self.index_z]
        
            #Estimation actuelle du rating
            prediction = g(self.u[:, self.user_id].dot(self.v[:,self.item_id]))
            
            #Estimation actuelle des links
            link_accuracy = g(self.u[:, self.user_id].dot(self.z[:,self.index_z]))
            
            #Ajout eventuel des biais
            if self.bias : 
                prediction += self.globalBias + self.usersBias[user_id] + self.itemsBias[item_id]

            diff_rating = (self.rating_id - prediction)
            diff_link = (self.link_id - link_accuracy)

            loss += abs(diff_rating)

            #Update des facteurs
            update_z = learning_rate*(
                   lambda_c*g_prime(self.u[:,self.user_id].dot(self.z[:,self.index_z]))*(-diff_link)*self.u[:,self.user_id] 
                 + lambda_z*self.z[:,self.index_z]
            )
            
            update_u = learning_rate*(
                (-diff_rating)*g_prime(self.u[:, self.user_id].dot(self.v[:,self.item_id]))*self.v[:,self.item_id] +
                lambda_c*(-diff_link)*g_prime(self.u[:, self.user_id].dot(self.z[:,self.index_z]))*self.z[:,self.index_z] +
                lambda_u*self.u[:, self.user_id]
            ) 
            
            update_v = learning_rate*(
                (-diff_rating)*g_prime(self.u[:, self.user_id].dot(self.v[:,self.item_id]))*self.u[:,self.user_id] +
                lambda_v*self.v[:, self.user_id]
            ) 
            
            self.z[:,self.index_z] -= update_z
            self.u[:,self.user_id] -= update_u
            self.v[:,self.item_id] -= update_v

            if self.bias:
                self.usersBias[user] = factor * self.usersBias[user] + learning_rate * diff
                self.itemsBias[item] = factor*self.itemsBias[item] + learning_rate * diff
                self.globalBias = factor * self.globalBias + learning_rate * diff

            self.loss_histo.append(loss)
            
            if(ep%int(nb_epochs/10)==0):
                accuracy_train = np.array([round(g(self.u[:,(users[i]-1)].dot(self.v[:,(items[i]-1)]))) == ratings[i] 
                                   for i in range(nbRatings)])
                print("Accuracy : %d : %f"%(ep, np.mean(accuracy_train)))
            
        print("Apprentissage du modele fini, accuracy final en train : %f"%(np.mean(accuracy_train)))
        
    def predict(self, userIds, itemIds):
        preds =[]
        for i in range(len(userIds)):   
            if itemIds[i] not in self.items :
                if userIds[i] not in self.users:
                    preds.append(np.random.randint(0,4))
                else : 
                    preds.append(round(g(np.random.rand(1,self.dimFactorisation).dot(self.v[:,itemIds[i]-1])[0])))
           
            elif userIds[i] not in self.users:
                preds.append(round(g(np.random.rand(1,self.dimFactorisation).dot(self.u[:,userIds[i]-1])[0])))

            else :
                preds.append(round(g(self.u[:,(userIds[i]-1)].dot(self.v[:,(itemIds[i]-1)]))))
        
        return np.array(preds)

In [225]:
user = np.array(movie['user_id']) 
item = np.array(movie['item_id']) 
ratings = np.array(movie['rating']) - 1
timestamps = np.array(movie['timestamp'])


user_train, user_test, item_train, item_test, ratings_train, ratings_test = train_test_split(user, item, ratings, train_size=0.8)
lambdada=0.6
r = 7

cf = SoRec(dimFactorisation=r)
cf.fit(user_train, item_train, ratings_train, m_links, nb_epochs=400*100*60,
       lambda_c=lambdada, lambda_u=lambdada, lambda_v=lambdada, lambda_z=lambdada,
       learning_rate=0.01
          )

#testons
print("L'accuracy en test est de  " +
      str(1*(cf.predict(user_test,item_test) == ratings_test).mean()))

print("La loss en test est de  " +
      str(np.abs(cf.predict(user_test,item_test) - ratings_test).mean()))

Accuracy : 0 : 0.164453
Accuracy : 240000 : 0.274391
Accuracy : 480000 : 0.303687
Accuracy : 720000 : 0.338953
Accuracy : 960000 : 0.360047
Accuracy : 1200000 : 0.373797
Accuracy : 1440000 : 0.378891
Accuracy : 1680000 : 0.381359
Accuracy : 1920000 : 0.384906
Accuracy : 2160000 : 0.384344
Apprentissage du modele fini, accuracy final en train : 0.384344
L'accuracy en test est de  0.329875
La loss en test est de  0.9235625
